In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier


In [2]:
def readfile(bestand):
    table = None
    table = pd.read_excel(bestand, index_col=0)
    table['time'] = table.index
    
    X = table.select_dtypes(include='number')
    y = pd.get_dummies(table.select_dtypes(exclude='number') )
    
    return X,y

In [3]:
def make_arr(XX):
    arr = pd.DataFrame(index=XX.index)
    arr['ARR_1'] = (XX['V0_measured']-XX['V1_measured']) - (XX['V1_measured']-XX['V2_measured'])
    
    dumV = XX['S'].diff().fillna(1).abs()*(XX['V0_measured']-XX['V2_measured'])
    dumV.replace(to_replace=0.0, inplace=True, method='ffill')
    
    dumT = XX['S'].diff().fillna(1).abs()*(XX.index)
    dumT[dumT.index==0.0] = 10E-10
    dumT.replace(to_replace=0.0, inplace=True, method='ffill')

    arr['ARR_2'] = (XX['V0_measured']-XX['V2_measured']) - dumV*np.exp(-(dumT.index - dumT)/2)
    
    return arr

In [4]:
# fit a RF model and determine gini importance

def selectRFmodel(XX,y):
#    imputer = KNNImputer(n_neighbors=3, weights="distance")
#    XX = imputer.fit_transform(XX)
    
    rfmodel = RandomForestClassifier(n_estimators = 100, criterion="gini",random_state = None)
    
    return rfmodel.fit(XX, y), rfmodel.feature_importances_

In [5]:
#load the balanced and complete training data in a pandas dataframe

#df_X, df_y = readfile("C:/Users/Admin/Pythonprojects/RAMS/data/Arduino_RRC_trainingset_05sec.xlsx")
df_X, df_y = readfile("https://raw.githubusercontent.com/chrisrijsdijk/RAMS/master/data/Arduino_RRC_trainingset_05sec.xlsx")

In [6]:
# infer RF models from df_X; df_y

X_names = df_X.columns      #load a string of column names to label the output of the random forest model afterwards...

#rf_X_H_model, rf_X_H_featimp = selectRFmodel(df_X, df_y['Health_state_H'])
#rf_X_R_model, rf_X_R_featimp = selectRFmodel(df_X, df_y['Health_state_R'])
#rf_X_C_model, rf_X_C_featimp = selectRFmodel(df_X, df_y['Health_state_C'])

rf_X_HRC_model, rf_X_HRC_featimp = selectRFmodel(df_X, df_y)

In [7]:
# infer RF models from arr's; df_y

df_as = make_arr(df_X)
as_names = df_as.columns      #load a string of column names to label the output of the random forest model afterwards...

#rf_A_H_model, rf_A_H_featimp = selectRFmodel(df_as, df_y['Health_state_H'])
#rf_A_R_model, rf_A_R_featimp = selectRFmodel(df_as, df_y['Health_state_R'])
#rf_A_C_model, rf_A_C_featimp = selectRFmodel(df_as, df_y['Health_state_C'])

rf_A_HRC_model, rf_A_HRC_featimp = selectRFmodel(df_as, df_y)

In [8]:
# infer RF models from X, arr's; df_y

df_Xas = pd.concat([df_X,df_as], axis='columns')
Xas_names = df_Xas.columns      #load a string of column names to label the output of the random forest model afterwards...

#rf_XA_H_model, rf_XA_H_featimp = selectRFmodel(df_Xas, df_y['Health_state_H'])
#rf_XA_R_model, rf_XA_R_featimp = selectRFmodel(df_Xas, df_y['Health_state_R'])
#rf_XA_C_model, rf_XA_C_featimp = selectRFmodel(df_Xas, df_y['Health_state_C'])

rf_XA_HRC_model, rf_XA_HRC_featimp = selectRFmodel(df_Xas, df_y)

In [9]:
#rf_XA_HRC_model.predict(df_Xas)
print('###########################################################################')
print(pd.DataFrame(rf_X_HRC_featimp, index=X_names, columns=['Gini importance of the model X']).sort_values(by='Gini importance of the model X', ascending=False) )
print(' ')
print(pd.DataFrame(rf_A_HRC_featimp, index=as_names, columns=['Gini importance of the model A']).sort_values(by='Gini importance of the model A', ascending=False) )
print(' ')
print(pd.DataFrame(rf_XA_HRC_featimp, index=Xas_names, columns=['Gini importance of the model XA']).sort_values(by='Gini importance of the model XA', ascending=False))
print('###########################################################################')

###########################################################################
             Gini importance of the model X
V1_measured                        0.539995
V2_measured                        0.313924
time                               0.112582
V0_measured                        0.025366
S                                  0.008133
 
       Gini importance of the model A
ARR_1                        0.536231
ARR_2                        0.463769
 
             Gini importance of the model XA
V1_measured                         0.286906
ARR_2                               0.235169
ARR_1                               0.212076
V2_measured                         0.173397
time                                0.042498
V0_measured                         0.030484
S                                   0.019470
###########################################################################


In [10]:
import serial
import time
import keyboard
from IPython.display import clear_output

In [11]:
# connect to Arduino

ser = serial.Serial('COM3', 9600, timeout=1) #check whether the Arduino is really connected with COM3 or adjust the code to the correct COM
                                             #check at settings and check whether Arduino is a connected device
time.sleep(2)

In [12]:
# read write the serial connection at a given sampling rate

def write_read(x):                  #this function takes an input x, which is a switch command and it returns data from the Arduino
    ser.write(bytes(x, 'utf-8'))    #pass the command to Arduino by the serial connection
    data = ser.readline()           #take data from Arduino
    return data                     #return the data to be processed further

In [13]:
# start to control the Arduino while collecting the data

diagnose = {}                                                    #just a dict to connect rf output to semantical fault labels
diagnose['[1 0 0]'] = 'Capacitance fault'
diagnose['[0 1 0]'] = 'Healthy'
diagnose['[0 0 1]'] = 'Resistance fault'
diagnose['[0 0 0]'] = '-'

diagnoses_X_HRC = []
diagnoses_A_HRC = []
diagnoses_XA_HRC = []

print('The switching period in the training set was 10 seconds')
print('Choose a switching period for the validation:')
try:
    period = int(input('Input:'))                                #state holding time of the switch
except ValueError:
    print("Not an integer")

num ='1'                                                         #num='1' is output to close the switch
value = write_read(num)                                          #sent out switch command and take sensor readings
        
t = 0.0                                                          #timer on the state holding time of the switch
tt = 0.0                                                         #timer on a switch cyle (on+off)
tijd = time.time()                                               #read current time
        
dummy = value.decode().rstrip().split()                          #process sensor readings
print(dummy)
dumV = (int(dummy[1])-int(dummy[3]))/1023*5                      #scale the voltages at the moment of a switch transition

dum4 = np.array([[int(dummy[0])]])                               #dum4 indicates the switch position
                                                                 #dum_X contains all sensor features switch position, voltages, time t+tt
dum_X = np.array([ np.append(dum4, np.append([int(j)/1023*5 for j in dummy[1:]],t+tt) ) ])

                                                                 #calculate arr1, arr2
arr1 = (  (int(dummy[1])-int(dummy[2])) - (int(dummy[2])-int(dummy[3])) )/1023*5
arr2 = (int(dummy[1])-int(dummy[3]))/1023*5 - dumV*np.exp(-t/2)
dum_A = np.array( [[arr1, arr2]] )                               #dum_A contains all arrs
dum_XA = np.array([np.append(dum_X, dum_A)])                     #merge dumX,dum_A to a unified set of explanatory variables

while not(keyboard.is_pressed('q')):                             #making a loop continues as long as i is true. This loop involves a cycle of reading and evaluating data
    clear_output(wait=True)
    print("The switching period during this validation is: ", period)
    print("Press <q> to quit")
    print("#########################################################")
          
    print('time:                                         ',str(pd.to_datetime(round(time.time(), 0),unit = 's')))
    print('.........................................................')                                     
    
    try:
        diagnoses_X_HRC  = diagnoses_X_HRC + [diagnose[str(rf_X_HRC_model.predict(dum_X)[0])]]
        diagnoses_A_HRC  = diagnoses_A_HRC + [diagnose[str(rf_A_HRC_model.predict(dum_A)[0])]]
        diagnoses_XA_HRC = diagnoses_XA_HRC + [diagnose[str(rf_XA_HRC_model.predict(dum_XA)[0])]]
        print('position of the switch S:                     ', dum_X[0][0])
        print('voltage V0:                                   ', dum_X[0][1])
        print('voltage V1:                                   ', dum_X[0][2])
        print('voltage V2:                                   ', dum_X[0][3])
        print('analytical redundancy relation 1:             ', arr1)
        print('analytical redundancy relation 2:             ', arr2)
        print('progress in the period [sec]:                 ',round(t,1))
        print('.........................................................')

        print('diagnostics by random forest classification from measurements:         ',diagnoses_X_HRC[-1] )
        print('diagnostics by random forest classification from arrs:                 ',diagnoses_A_HRC[-1] )
        print('diagnostics by random forest classification from measurements and arrs:',diagnoses_XA_HRC[-1] )
        print('.........................................................')

        print('frequencies of random forest classification from measurements:        ',
              str(np.unique(diagnoses_X_HRC, return_counts=True)[0]), 
              str(np.unique(diagnoses_X_HRC, return_counts=True)[1]) )
        
        print('frequencies of random forest classification from arrs                 ',
              str(np.unique(diagnoses_A_HRC, return_counts=True)[0]), 
              str(np.unique(diagnoses_A_HRC, return_counts=True)[1]) )

        print('frequencies of random forest classification from measurements and arrs',
              str(np.unique(diagnoses_XA_HRC, return_counts=True)[0]), 
              str(np.unique(diagnoses_XA_HRC, return_counts=True)[1]) )        
        
    except:
        print(' ')
              
    if t < period:                                               #check t<the entered state holding time of the switch
        value = write_read(num)                                  #take the sensorreading while continuing the switch position num
        
        t = time.time()-tijd                                     #refresh t
        
        dummy = value.decode().rstrip().split()                  #process sensor readings
        
        dum4 = np.array([[int(dummy[0])]])                       #dum4 indicates the switch position
                                                                 #dum_X contains all sensor features switch position, voltages, time t+tt        
        dum_X = np.array([ np.append(dum4, np.append([int(j)/1023*5 for j in dummy[1:]],t+tt) ) ])
                                                                 #calculate arr1, arr2        
        arr1 = (  (int(dummy[1])-int(dummy[2])) - (int(dummy[2])-int(dummy[3])) )/1023*5
        arr2 = (int(dummy[1])-int(dummy[3]))/1023*5 - dumV*np.exp(-t/2)
        dum_A = np.array( [[arr1, arr2]] )                       #dum_A contains all arrs
        dum_XA = np.array([np.append(dum_X, dum_A)])             #merge dumX,dum_A to a unified set of explanatoryu variables
        
    else:
        num = str( abs(int(num) - 1) )                           #flip the switch position from zero to one or conversely
        value = write_read(num)                                  #take the sensorreading upon the switch transition
        
        t = 0.0                                                  #reset the timer on the state holding time
        tt = int(num) * t                                        #refresh the timer on the switching cycle
        tijd = time.time()                                       #refresh tijd of the last switch transition
        
        dummy = value.decode().rstrip().split()                  #process sensor readings

        dumV = (int(dummy[1])-int(dummy[3]))/1023*5              #scale the voltages at the moment of a switch transition
        
        dum4 = np.array([[int(dummy[0])]])                       #dum4 indicates the switch position
                                                                 #dum_X contains all sensor features switch position, voltages, time t+tt        
        dum_X = np.array([ np.append(dum4, np.append([int(j)/1023*5 for j in dummy[1:]],t+tt) ) ])
                                                                 #calculate arr1, arr2        
        arr1 = (  (int(dummy[1])-int(dummy[2])) - (int(dummy[2])-int(dummy[3])) )/1023*5
        arr2 = (int(dummy[1])-int(dummy[3]))/1023*5 - dumV*np.exp(-t/2)
        dum_A = np.array( [[arr1, arr2]] )                       #dum_A contains all arrs
        dum_XA = np.array([np.append(dum_X, dum_A)])             #merge dumX,dum_A to a unified set of explanatoryu variables
    
print('You pressed <q>uit')
ser.close()       #close the serial port when the reading is stopped. If not done, reconnection with the Arduino is not possible

The switching period during this validation is:  3
Press <q> to quit
#########################################################
time:                                          2024-10-07 07:54:42
.........................................................
position of the switch S:                      0.0
voltage V0:                                    0.0
voltage V1:                                    1.2903225806451613
voltage V2:                                    2.590420332355816
analytical redundancy relation 1:              0.009775171065493646
analytical redundancy relation 2:              -0.013011229560717652
progress in the period [sec]:                  1.1
.........................................................
diagnostics by random forest classification from measurements:          Healthy
diagnostics by random forest classification from arrs:                  Healthy
diagnostics by random forest classification from measurements and arrs: Healthy
.............................

In [14]:
ser.close()